In [ ]:
!pip -q install ultralytics

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 715.0/715.0 kB 4.9 MB/s eta 0:00:00


In [ ]:
!wget https://the-eye.eu/public/AI/cah/laion400m-met-release/laion400m-meta/part-00000-5b54c5d5-bbcf-484d-a2ce-0d6f73df1a36-c000.snappy.parquet

--2024-02-16 22:27:22--  https://the-eye.eu/public/AI/cah/laion400m-met-release/laion400m-meta/part-00000-5b54c5d5-bbcf-484d-a2ce-0d6f73df1a36-c000.snappy.parquet
Resolving the-eye.eu (the-eye.eu)... 162.213.130.6
Connecting to the-eye.eu (the-eye.eu)|162.213.130.6|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1803288490 (1.7G) [application/octet-stream]
Saving to: ‘part-00000-5b54c5d5-bbcf-484d-a2ce-0d6f73df1a36-c000.snappy.parquet’

part-00000-5b54c5d5 100%[===================>]   1.68G  10.8MB/s    in 3m 13s  

2024-02-16 22:30:35 (8.92 MB/s) - ‘part-00000-5b54c5d5-bbcf-484d-a2ce-0d6f73df1a36-c000.snappy.parquet’ saved [1803288490/1803288490]



In [ ]:
import json
import pandas as pd
import numpy as np
import urllib3
from tqdm import trange
import os
import torch
from ultralytics import YOLO
from collections import Counter
from scipy import stats
import requests
from datetime import datetime


In [ ]:
metadata = pd.read_parquet("/content/part-00000-5b54c5d5-bbcf-484d-a2ce-0d6f73df1a36-c000.snappy.parquet")
print(metadata.shape)
metadata = metadata[2000000:4000000]

(12933524, 8)


In [ ]:
model = YOLO("yolov8n.pt")
IMG_FORMATS = ['bmp', 'dng', 'jpeg', 'jpg', 'mpo', 'png', 'tif', 'tiff', 'webp', 'pfm']
urls_raw = list(metadata["URL"])
caps_raw = list(metadata["TEXT"])

100%|██████████| 6.23M/6.23M [00:00<00:00, 78.6MB/s]


In [ ]:
metadata[0:1]

,SAMPLE_ID,URL,TEXT,HEIGHT,WIDTH,LICENSE,NSFW,similarity
2000000,1.311530e+11,https://www.theroyalforums.com/forums/attachme...,Click image for larger version,NaN,NaN,None,None,NaN


In [ ]:
import re
def cap_parse(cap):
    nums = []
    num2word = {
        'two':2,
        'three':3,
        'four':4,
        'five':5,
        'six':6,
        'seven':7,
        'eight':8,
        'nine':9,
        'ten':10
    }
    try:
        x = re.findall(r'\b(two|three|four|five|six|seven|eight|nine|ten)\b', cap, flags=re.IGNORECASE)
    except:
        return (False,-1)
    if len(x) == 0 :return (False,-1)
    else:
        max_freq = max(num2word[i.lower()] for i in x)
        return (True,max_freq)



txt0 = "vintage silver plate tablespoons, serving spoon set of six 1847 Rogers pattern"
txt1 = "A preview of seven printable dot marker coloring pages. Each page has a large 4th of July themed image with dots to color in with a dauber style marker."
txt2 = "bag of chips"
cap_parse(txt2)


(False, -1)

In [ ]:

def validate_urls(urls,caps):
    valid_urls = []
    valid_caps = []
    for url,cap in zip(urls,caps):
        for img_format in IMG_FORMATS:
            idx = url.find("." + img_format)
            if idx == -1:
                continue
            valid_urls.append(url[:idx + len(img_format) + 1])
            valid_caps.append(cap)
    return (valid_urls,valid_caps)




In [ ]:
def validate_counts(counting):
    final = []
    for i in range(len(counting)):
        print(counting[i])
        x,cap,url,num = counting[i]
        if len(x)!=0:
            counts = Counter(map(int, x))
            max_freq = max(counts.values())
            print(max_freq)
            if max_freq == num:
                final.append(counting[i])
    return final


In [ ]:
a = [([torch.tensor(39., dtype=int), torch.tensor(39., dtype=int)],"aa","bb",3),([39,39,39],"aa","bb",2)]
final = validate_counts(a)
final
# a[0]
# x,cap,url,num = a[0]

([tensor(39), tensor(39)], 'aa', 'bb', 3)
2
([39, 39, 39], 'aa', 'bb', 2)
3


[]

In [ ]:
validate_counts([([torch.tensor(32), torch.tensor(32), torch.tensor(0)], 'http://s3./bncore/wp-conten.jpg', 'Matthew Anderson', 5)])

([tensor(32), tensor(32), tensor(0)], 'http://s3./bncore/wp-conten.jpg', 'Matthew Anderson', 5)
2


[]

In [ ]:
def save_counts(counting):
    print(len(counting))
    counting[0:5]
    f = open("segments4.npy","wb")
    np.save(f,pd.DataFrame(counting))
    print("counting set 4 saved")


In [ ]:

def filter_laion(urls_raw,caps_raw):
    urls,caps = validate_urls(urls_raw,caps_raw)
    counting = []
    timeout = 1
    print(len(urls))
    for i in trange(len(urls)):
        # blockPrint()
        cap_data = cap_parse(caps[i])
        # print(cap_data,caps[i])
        if cap_data[0] == True:
            try:
                start_time = datetime.now()
                response = requests.get(urls[i], timeout=timeout)
                end_time = datetime.now()
                time_taken = (end_time - start_time).total_seconds()

                if response.status_code == 200 and time_taken <= timeout:
                  results = model(str(urls[i]),verbose=False,stream=False)
                  x = list(results[0].boxes.cls)
                else:
                  x=[]
            except:
                x = []
            counting.append((x,urls[i],caps[i],cap_data[1]))
        # enablePrint()
    counting = validate_counts(counting)
    save_counts(counting)
    return counting



In [ ]:
filter_laion(urls_raw,caps_raw)


1827613


  0%|          | 64/1827613 [00:02<19:09:19, 26.50it/s]


100%|██████████| 22.0k/22.0k [00:00<00:00, 26.7MB/s]
  0%|          | 108/1827613 [00:06<33:27:35, 15.17it/s]


100%|██████████| 5.91k/5.91k [00:00<00:00, 7.29MB/s]
  0%|          | 175/1827613 [00:07<18:21:35, 27.65it/s]


100%|██████████| 12.0k/12.0k [00:00<00:00, 27.0MB/s]
  0%|          | 179/1827613 [00:08<20:19:30, 24.98it/s]

...


100%|██████████| 22.1k/22.1k [00:00<00:00, 47.7MB/s]
 27%|██▋       | 500918/1827613 [1:59:45<6:20:34, 58.10it/s]


100%|██████████| 48.1k/48.1k [00:00<00:00, 40.8MB/s]
 27%|██▋       | 500931/1827613 [1:59:46<9:54:54, 37.17it/s]


100%|██████████| 29.7k/29.7k [00:00<00:00, 32.4MB/s]
 27%|██▋       | 501146/1827613 [1:59:47<4:22:47, 84.13it/s]


100%|██████████| 208k/208k [00:00<00:00, 2.70MB/s]
 27%|██▋       | 501166/1827613 [1:59:49<6:28:49, 56.86it/s]


100%|██████████| 31.4k/31.4k [00:00<00:00, 36.4MB/s]

In [ ]:
f = open("segments4.npy","rb")
out = np.load(f,allow_pickle=True)
len(out)


In [ ]:
from google.colab import files
files.download('segments4.npy')

In [ ]:
out

In [ ]:
validate_counts([(list([torch.tensor(39.),torch.tensor(39.),torch.tensor(39.)]),"aa","bb",3)])

In [ ]:
import requests

def is_url_image(image_url):
   image_formats = ("image/png", "image/jpeg", "image/jpg")
   try:
      r = requests.head(image_url)
   except:
      return False
   try:
      if r.headers["content-type"] in image_formats:
         return True
   except:
         return True

   return False